# Laboratorio: El ADN de un Negocio Exitoso con Datos de Yelp 🚀

¡Muy buenos días, futuros consultores y estrategas de negocio!

Hoy nos quitamos el sombrero de estudiantes y nos ponemos el de **consultores de datos**. Nuestro nuevo cliente, un emprendedor entusiasta, quiere abrir un restaurante en una ciudad muy competitiva y nos ha contratado para que le demos una ventaja. Nuestra misión: **usar el poder de los datos de Yelp para descubrir la fórmula secreta del éxito.**

No vamos a programar desde cero. Vamos a usar el código como si fuera un potente escáner: lo ejecutaremos para analizar la evidencia y nuestra verdadera habilidad será interpretar los resultados y dar recomendaciones estratégicas. ¡Empecemos!

In [ ]:
# Este bloque instala librerías y descarga el set de datos de Yelp.
# El archivo es grande, así que este paso puede tardar entre 2 y 5 minutos. ¡Paciencia!

print("⚙️ Instalando librerías necesarias...")
!pip install wordcloud -q

print("📥 Descargando el set de datos de Yelp (esto puede tardar)...")
# Usaremos una versión alojada en Kaggle para facilitar la descarga
!wget -q --show-progress https://storage.googleapis.com/big-data-course-assets/yelp_dataset.tar

print("\n📦 Descomprimiendo archivos...")
!tar -xf yelp_dataset.tar

print("\n✅ ¡Entorno listo! Ya tenemos los datos para empezar nuestra investigación.")

In [ ]:
# Vamos a cargar los datos de los negocios y las reseñas.
# Pandas es nuestra herramienta para manejar estas tablas de datos.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

# --- Carga de Datos ---
# Los archivos son 'JSON por línea', por eso usamos lines=True.
print("Cargando datos de negocios...")
df_business = pd.read_json('yelp_academic_dataset_business.json', lines=True)

print("Cargando datos de reseñas (esto puede tomar un momento)...")
# Para que el notebook sea ágil, solo cargaremos las primeras 2 millones de reseñas.
df_review = pd.read_json('yelp_academic_dataset_review.json', lines=True, chunksize=2000000).get_chunk()


# --- Primera Mirada ---
print("\n--- Vista Previa de los Datos de Negocios ---")
print(f"Tenemos datos de {df_business.shape[0]} negocios.")
display(df_business.head())

print("\n--- Vista Previa de los Datos de Reseñas ---")
print(f"Tenemos {df_review.shape[0]:,} reseñas para analizar.")
display(df_review.head())

## PARTE 1: El Panorama General (Análisis de Datos Estructurados) 🏙️

Todo buen consultor empieza con una vista panorámica. Antes de analizar un negocio, debemos entender el mercado. Usaremos la ciudad de **Las Vegas** como nuestro caso de estudio. La información en `df_business` (estrellas, cantidad de reseñas, ciudad) es **dato estructurado**: ordenado en filas y columnas, fácil de contar y agregar.

In [ ]:
# Filtramos la tabla para quedarnos solo con los negocios en Las Vegas.
df_vegas = df_business[df_business['city'] == 'Las Vegas'].copy()

print(f"Análisis de mercado: Hay {df_vegas.shape[0]} negocios registrados en Las Vegas en nuestro dataset.")

# Graficamos la distribución de las calificaciones (estrellas)
plt.figure(figsize=(10, 6))
sns.countplot(x='stars', data=df_vegas, palette='viridis', order=df_vegas['stars'].value_counts().index)
plt.title('Distribución de Calificaciones (Estrellas) en Las Vegas', fontsize=16)
plt.xlabel('Calificación (Estrellas)', fontsize=12)
plt.ylabel('Cantidad de Negocios', fontsize=12)
plt.show()

### 🕵️ Mini Reto #1: ¿Dónde está la competencia?

Nuestro cliente quiere abrir un restaurante. ¿Cuáles son las categorías de negocio más comunes en Las Vegas? ¿Y las más saturadas?

**Su Tarea:** Ejecute el siguiente código e interprete el gráfico para responder:
1.  ¿Cuál es la categoría de negocio más común en Las Vegas?
2.  Si quisiera abrir un negocio con menos competencia directa, ¿qué tipo de negocio evitaría según este gráfico?

In [ ]:
# Este código procesa la columna 'categories', que es un texto, para contar cada categoría individual.
df_vegas.dropna(subset=['categories'], inplace=True)
categories_exploded = df_vegas['categories'].str.split(', ').explode()

plt.figure(figsize=(12, 8))
categories_exploded.value_counts().nlargest(15).sort_values().plot(kind='barh')
plt.title('Top 15 Categorías de Negocio Más Comunes en Las Vegas', fontsize=16)
plt.xlabel('Cantidad de Negocios', fontsize=12)
plt.show()

## PARTE 2: Los Detalles que Importan (Análisis de Datos Semi-Estructurados) 📋

Ahora, profundicemos. ¿Qué hace que un negocio tenga éxito más allá de su categoría? La columna `attributes` contiene información valiosísima (si aceptan tarjeta, si tienen terraza, etc.), pero no está en una columna ordenada. Es un **dato semi-estructurado** (un objeto JSON anidado). Nuestra tarea es "desempacarlo" para analizarlo.

### 🤔 Mini Reto #2: ¿Pagar por el ambiente vale la pena?

El cliente se pregunta si debe invertir en una terraza (`OutdoorSeating`). ¿Acaso los restaurantes con terraza tienen, en promedio, una mejor calificación que los que no la tienen?

**Su Tarea:** Ejecute el código de abajo, que extrae el atributo `OutdoorSeating` y calcula la calificación promedio para ambos grupos. Responda:
1.  ¿Cuál es la calificación promedio de los restaurantes con y sin terraza?
2.  Basado **únicamente** en estos datos, ¿le recomendaría al cliente invertir en una terraza? ¿Por qué?

In [ ]:
# 1. Filtramos para quedarnos solo con restaurantes
df_restaurants = df_vegas[df_vegas['categories'].str.contains('Restaurants', na=False)].copy()

# 2. Extraemos el atributo 'OutdoorSeating' del JSON
# No se preocupe por el código, enfóquese en el resultado
attributes_dict = df_restaurants['attributes'].dropna().apply(json.loads)
df_attributes = pd.json_normalize(attributes_dict)
df_restaurants_merged = df_restaurants.join(df_attributes)

# 3. Calculamos y mostramos la calificación promedio
# Convertimos el valor a booleano (True/False) para el análisis
df_restaurants_merged['OutdoorSeating'] = df_restaurants_merged['OutdoorSeating'].apply(lambda x: x == 'True' if isinstance(x, str) else None)
avg_stars_seating = df_restaurants_merged.groupby('OutdoorSeating')['stars'].mean().round(2)

print("--- Calificación Promedio según si tienen Terraza ---")
print(avg_stars_seating)

## PARTE 3: La Voz del Cliente (Análisis de Datos No Estructurados - Texto) 💬

Los números y atributos nos dicen 'qué' pasa, pero no 'por qué'. Para eso, necesitamos analizar el **dato no estructurado** más valioso que tenemos: las reseñas de los clientes. Vamos a comparar las palabras más usadas en las reseñas de 1 estrella contra las de 5 estrellas para encontrar los secretos del éxito y las causas del fracaso.

### 🗣️ Mini Reto #3: La anatomía del éxito y el fracaso

**Su Tarea:** Ejecute los dos bloques de código siguientes para generar las "nubes de palabras" de las reseñas. Una para las peores experiencias (1 estrella) y otra para las mejores (5 estrellas). Luego, compare las dos imágenes y responda:

1.  ¿Cuáles son las 3 palabras más destacadas en las reseñas de 5 estrellas? ¿Qué nos dice esto sobre lo que más valoran los clientes satisfechos?
2.  ¿Cuáles son las 3 palabras más destacadas en las reseñas de 1 estrella? ¿Qué nos dice esto sobre los errores más comunes que cometen los negocios?
3.  ¿Qué recomendación clave sobre el **servicio al cliente** le daría al emprendedor basándose en esta comparación?

In [ ]:
from wordcloud import WordCloud

# Unimos los dataframes para tener las reseñas con la info del negocio
df_vegas_reviews = pd.merge(df_vegas, df_review, on='business_id', how='inner')

# Creamos un gran texto con todas las reseñas de 5 estrellas
text_5_star = " ".join(review for review in df_vegas_reviews[df_vegas_reviews['stars_y'] == 5]['text'])

# Generamos la nube de palabras
wordcloud_5_star = WordCloud(background_color="white", max_words=50, colormap='viridis').generate(text_5_star)

plt.figure(figsize=(12, 8))
plt.imshow(wordcloud_5_star, interpolation='bilinear')
plt.axis("off")
plt.title("Palabras Más Comunes en Reseñas de 5 Estrellas", fontsize=20)
plt.show()

In [ ]:
# Creamos un gran texto con todas las reseñas de 1 estrella
text_1_star = " ".join(review for review in df_vegas_reviews[df_vegas_reviews['stars_y'] == 1]['text'])

# Generamos la nube de palabras
wordcloud_1_star = WordCloud(background_color="black", max_words=50, colormap='Reds').generate(text_1_star)

plt.figure(figsize=(12, 8))
plt.imshow(wordcloud_1_star, interpolation='bilinear')
plt.axis("off")
plt.title("Palabras Más Comunes en Reseñas de 1 Estrella", fontsize=20)
plt.show()

## 🏆 El Reto Final: Su Recomendación Estratégica

Ha analizado el mercado, los atributos del negocio y la voz del cliente. Ahora es el momento de actuar como un verdadero consultor.

Nuestro cliente quiere abrir un restaurante de **Comida Peruana** en Las Vegas. Basándose en todo el análisis que ha ejecutado en este notebook:

**Su Tarea Final:** Elabore una recomendación de 3 puntos clave para el cliente. Sea breve y directo.

1.  **La Oportunidad de Mercado:** ¿Recomendaría abrir un restaurante peruano? ¿Por qué? (Puede volver a ejecutar el código del Reto #1 filtrando por "Peruvian" o "Restaurants" para justificar su respuesta).
2.  **La Clave del Servicio (Basado en el Reto #3):** ¿Cuál es el factor #1 relacionado con el servicio o la experiencia que debe obsesionar al cliente para conseguir reseñas de 5 estrellas?
3.  **Una Inversión Inteligente (Basado en el Reto #2):** Mencione un atributo o característica del local (como la terraza u otro que pueda explorar) que los datos sugieren que tiene un impacto positivo en la calificación.

Prepare sus respuestas. En nuestra próxima sesión, las discutiremos en grupo. ¡Excelente trabajo, consultores!